In [25]:
pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/09/05/28664524fcc67c078313d482bf25fe403e9399130622cfc89e185ec0abf6/yfinance-0.2.54-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/04/13/d9839089b900fa7b479cce495d62110cddc4bd5630a04d8469916c0e79c5/frozendict-2.4.6-py311-none-any.whl.metadata
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---- ----------------------------------- 0.4/3.0 MB 11.2 MB/s eta 0:00:01
     -------------- ------------------------- 1.1/3.0 MB 13.9 MB/s eta 0:00:01
     -------------------------- ------------- 2.0/3.0 MB 16.2 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 M


[notice] A new release of pip is available: 23.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import pandas as pd
import sqlite3
import os
import yfinance as yf

In [27]:
tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NVDA", "BRK-B", "META", "JNJ", "V", "XOM", "UNH", "PG", "HD", "MA", "CVX", "PEP", "KO", "LLY", "ABBV"]
tickers_str = " ".join(tickers)

In [34]:
tickers_data = yf.download(tickers_str, start="2020-01-01", end="2020-01-05",  actions=True, group_by="ticker")

[*********************100%***********************]  20 of 20 completed


In [40]:
data_list = []
for ticker in tickers_data.columns.levels[0]:
    # Selecionar o DataFrame para cada ticker
    df_ticker = tickers_data[ticker].copy()
    
    # Adicionar o ticker como uma nova coluna
    df_ticker['Ticker'] = ticker
    
    # Resetar o índice para incluir as datas como uma coluna
    df_ticker = df_ticker.reset_index()
    
    # Adicionar o DataFrame ajustado à lista
    data_list.append(df_ticker)

In [41]:
df_ticker

Price,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
0,2020-01-02,54.624698,55.231292,54.624698,55.137974,12456400,0.0,0.0,XOM
1,2020-01-03,55.480161,55.503497,54.562497,54.694702,17386900,0.0,0.0,XOM


In [2]:
conn = sqlite3.connect(r"../../../data/database/financial_data.db")

In [3]:
with open("close_window_30_90_180.sql", 'r', encoding='utf-8') as file:
    window_query = file.read().strip() 
with open("pct_change.sql", 'r', encoding='utf-8') as file:
    pct_query = file.read().strip() 

In [38]:
df = pd.read_sql("SELECT * FROM assets;", conn)
df_mean_window = pd.read_sql(window_query, conn)
df_pct_change = pd.read_sql(window_query, conn)

In [39]:
df['id']

0            1
1            2
2            3
3            4
4            5
         ...  
38555    38556
38556    38557
38557    38558
38558    38559
38559    38560
Name: id, Length: 38560, dtype: int64

In [14]:
df

,id,ticker,date,high,open,low,close,volume,dividends,stock_splits
0,1,AAPL,2020-01-02,72.856621,71.799881,71.545395,72.796028,135480400.0,0.0,0.0
1,2,AAPL,2020-01-03,72.851761,72.020432,71.862892,72.088295,146322800.0,0.0,0.0
2,3,AAPL,2020-01-06,72.701492,71.206070,70.954002,72.662712,118387200.0,0.0,0.0
3,4,AAPL,2020-01-07,72.929322,72.672409,72.100418,72.320976,108872000.0,0.0,0.0
4,5,AAPL,2020-01-08,73.787315,72.022858,72.022858,73.484352,132079200.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
38555,38556,PG,2025-02-14,170.000000,169.789993,162.300003,162.889999,13172700.0,0.0,0.0
38556,38557,TSLA,2025-02-14,362.000000,360.619995,347.500000,355.839996,68054700.0,0.0,0.0
38557,38558,UNH,2025-02-14,533.200012,529.000000,522.909973,523.510010,3703700.0,0.0,0.0
38558,38559,V,2025-02-14,355.429993,354.429993,353.220001,353.809998,5719100.0,0.0,0.0


In [10]:
df_mean_window

,date,ticker,mean_30,mean_90,mean_180
0,2020-01-02,AAPL,NaN,NaN,NaN
1,2020-01-03,AAPL,NaN,NaN,NaN
2,2020-01-06,AAPL,NaN,NaN,NaN
3,2020-01-07,AAPL,NaN,NaN,NaN
4,2020-01-08,AAPL,NaN,NaN,NaN
...,...,...,...,...,...
38555,2025-02-14,XOM,108.179675,108.052635,109.230771
38556,2025-02-14,XOM,108.183223,108.062415,109.211821
38557,2025-02-14,XOM,108.186772,108.072195,109.192871
38558,2025-02-14,XOM,108.190320,108.081975,109.173921


In [15]:
print(df.duplicated(subset=['date', 'ticker']).sum())  # Verifica duplicatas em df

12800


In [19]:
df_duplicates = df[df.duplicated(subset=['date', 'ticker'], keep=False)]
df_mean_duplicates = df_mean_window[df_mean_window.duplicated(subset=['date', 'ticker'], keep=False)]

In [23]:
df_duplicates.sort_values(by=['ticker','date'])

,id,ticker,date,high,open,low,close,volume,dividends,stock_splits
1267,1268,AAPL,2025-01-16,238.009995,237.350006,228.029999,228.259995,71759100.0,0.0,0.0
25360,25361,AAPL,2025-01-16,238.009995,237.350006,228.029999,228.259995,71759100.0,0.0,0.0
25380,25381,AAPL,2025-01-16,238.009995,237.350006,228.029999,228.259995,71759100.0,0.0,0.0
25400,25401,AAPL,2025-01-16,238.009995,237.350006,228.029999,228.259995,71759100.0,0.0,0.0
25420,25421,AAPL,2025-01-16,238.009995,237.350006,228.029999,228.259995,71759100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
38479,38480,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0
38499,38500,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0
38519,38520,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0
38539,38540,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0


In [21]:
df_mean_duplicates

,date,ticker,mean_30,mean_90,mean_180
1267,2025-01-16,AAPL,245.921290,233.725158,221.399768
1268,2025-01-16,AAPL,245.555806,233.809583,221.705765
1269,2025-01-16,AAPL,245.091612,233.893020,222.029213
1270,2025-01-16,AAPL,244.615806,233.985238,222.358332
1271,2025-01-16,AAPL,244.139032,234.049466,222.666916
...,...,...,...,...,...
38555,2025-02-14,XOM,108.179675,108.052635,109.230771
38556,2025-02-14,XOM,108.183223,108.062415,109.211821
38557,2025-02-14,XOM,108.186772,108.072195,109.192871
38558,2025-02-14,XOM,108.190320,108.081975,109.173921


In [17]:
print(df[['date', 'ticker']].nunique())  # Número de combinações únicas em df
print(df_mean_window[['date', 'ticker']].nunique())  # Número de combinações únicas em df_mean_window


date      1288
ticker      20
dtype: int64
date      1288
ticker      20
dtype: int64


In [11]:
feature_table = df.merge(df_mean_window, on=['date','ticker'], how='inner')

In [12]:
feature_table

,id,ticker,date,high,open,low,close,volume,dividends,stock_splits,mean_30,mean_90,mean_180
0,1,AAPL,2020-01-02,72.856621,71.799881,71.545395,72.796028,135480400.0,0.0,0.0,NaN,NaN,NaN
1,2,AAPL,2020-01-03,72.851761,72.020432,71.862892,72.088295,146322800.0,0.0,0.0,NaN,NaN,NaN
2,3,AAPL,2020-01-06,72.701492,71.206070,70.954002,72.662712,118387200.0,0.0,0.0,NaN,NaN,NaN
3,4,AAPL,2020-01-07,72.929322,72.672409,72.100418,72.320976,108872000.0,0.0,0.0,NaN,NaN,NaN
4,5,AAPL,2020-01-08,73.787315,72.022858,72.022858,73.484352,132079200.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632595,38560,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0,108.179675,108.052635,109.230771
3632596,38560,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0,108.183223,108.062415,109.211821
3632597,38560,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0,108.186772,108.072195,109.192871
3632598,38560,XOM,2025-02-14,110.230003,108.519997,108.099998,108.239998,15726700.0,0.0,0.0,108.190320,108.081975,109.173921


In [7]:
df_pct_change

,date,ticker,mean_30,mean_90,mean_180
0,2020-01-02,AAPL,NaN,NaN,NaN
1,2020-01-03,AAPL,NaN,NaN,NaN
2,2020-01-06,AAPL,NaN,NaN,NaN
3,2020-01-07,AAPL,NaN,NaN,NaN
4,2020-01-08,AAPL,NaN,NaN,NaN
...,...,...,...,...,...
38555,2025-02-14,XOM,108.179675,108.052635,109.230771
38556,2025-02-14,XOM,108.183223,108.062415,109.211821
38557,2025-02-14,XOM,108.186772,108.072195,109.192871
38558,2025-02-14,XOM,108.190320,108.081975,109.173921


In [ ]:
feature_table.merge(df_pct_change, on=['date','ticker'])